In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

Get the list of states

In [6]:
def getStates():
   states = []

   url = "https://mausam.imd.gov.in/imd_latest/contents/agromet/advisory/indiadistrictforecast.php"
   driver = webdriver.Chrome()
   driver.get(url)

   state_dd_class= (By.CLASS_NAME, "myselect")
   state_dd_elem= WebDriverWait(driver, 10).until(EC.presence_of_element_located(state_dd_class))
   state_selection = Select(state_dd_elem)


   for opt in state_selection.options:
      states.append(opt.text)

   driver.quit()

   states = states[1:]
   return states

states = getStates()

Store the list of states

In [7]:
import csv
file = open('List_of_states.csv', 'w', newline = '')
w = csv.writer(file)

for i in states:
    w.writerow([i])

file.close()

Import list of states

In [8]:
import csv
file = open('List_of_states.csv', 'r')
c = csv.reader(file)

states = []
for i in c:
    states.append(i[0].strip())

file.close()

Get district names for a state

In [10]:
def getDistrict(state):
    districts = []

    url = "https://mausam.imd.gov.in/imd_latest/contents/agromet/advisory/indiadistrictforecast.php"
    driver = webdriver.Chrome()
    driver.get(url)

    state_dd_class= (By.CLASS_NAME, "myselect")
    state_dd_elem= WebDriverWait(driver, 10).until(EC.presence_of_element_located(state_dd_class))
    state_selection = Select(state_dd_elem)
    state_selection.select_by_value(state)  


    district_dd_class= (By.CLASS_NAME, "myselect1")  
    district_dd_elem =WebDriverWait(driver, 10).until(EC.presence_of_element_located(district_dd_class))
    district_selection = Select(district_dd_elem)

    districts = []

    for opt in district_selection.options:
       districts.append(opt.text)

    driver.quit()

    districts = districts[1:]

    return districts

districts = getDistrict('WEST BENGAL')

Store list of districts per state

In [12]:
import csv

file = open('List_of_districts_per_state.csv', 'w', newline='')
w = csv.writer(file)

for i in states:
    dis = getDistrict(i)
    l = [i] + dis
    w.writerow(l)

file.close()

Import district wise list

In [13]:
states = []
dis = {}

import csv

file = open('List_of_districts_per_state.csv', 'r')
c = csv.reader(file)

for i in c:
    s = i[0]
    states.append(s)
    dis[s] = []

    for j in i[1:]:
        dis[s].append(j)

file.close()

Get the table given state and district

In [22]:
def getTable(state, district):
    url = "https://mausam.imd.gov.in/imd_latest/contents/agromet/advisory/indiadistrictforecast.php"
    driver = webdriver.Chrome()
    driver.get(url)

    state_dd_class= (By.CLASS_NAME, "myselect")
    state_dd_elem= WebDriverWait(driver, 10).until(EC.presence_of_element_located(state_dd_class))
    state_selection = Select(state_dd_elem)
    state_selection.select_by_value(state)  

    district_dd_class= (By.CLASS_NAME, "myselect1")  
    district_dd_elem =WebDriverWait(driver, 10).until(EC.presence_of_element_located(district_dd_class))
    district_selection = Select(district_dd_elem)
    district_selection.select_by_value(district)  

    table_class =(By.XPATH, "//table")
    table_elem= WebDriverWait(driver, 10).until(EC.presence_of_element_located(table_class))

    table_html= table_elem.get_attribute("outerHTML")
    soup = BeautifulSoup(table_html, 'html.parser')
    rows= soup.find_all('tr')

    header_elems = rows[0].find_all('th')
    data= []
    headers= []
    for th in header_elems:
        text= th.text.strip()
        headers.append(text)

    data.append(headers)

    for row in rows[1:]:
        row_data =[]
        for td in row.find_all('td'):
            text= td.text.strip()
            row_data.append(text)
        data.append(row_data)

    driver.quit()

    import pandas as pd
    table = []

    for row in range(len(data)):
        l = []
        r = data[row]
        if (row == 0):
            l.append(r[0])
            l.append('Valid From')
            l.append('Valid Till')
            for j in range(2, len(r)):
                l.append(r[j])
        else:
            x = r[1]
            s = x.split('OF')
            fr = s[1].split('TO')[0].strip()
            to = s[2].strip()

            l.append(r[0])
            l.append(fr)
            l.append(to)
            for j in range(2, len(r)):
                l.append(r[j])
                
        table.append(l)

    return table

In [16]:
file = open('log.txt', 'w')
s = 'tathagata'

for i in s:
    file.write(i)
    file.write('\n')
file.close()

In [33]:
file = open('log.txt', 'r')
l = []
s = file.readline()

if s == '':
    print('Nothing')
    
while s:
    l.append(s.strip())
    s = file.readline()
file.close()

print(l)

['Andaman & Nicobar Island,N-AND-M-ANDAMAN', 'Andaman & Nicobar Island,NICOBAR', 'Andaman & Nicobar Island,SOUTH-ANDAMANS', 'Andhra Pradesh,ANANTPUR', 'Andhra Pradesh,CHITTOOR', 'Andaman & Nicobar Island,N-AND-M-ANDAMAN', 'Andaman & Nicobar Island,NICOBAR', 'Andaman & Nicobar Island,SOUTH-ANDAMANS', 'Andhra Pradesh,ANANTPUR', 'Andhra Pradesh,CHITTOOR', 'Andaman & Nicobar Island,N-AND-M-ANDAMAN']


Start scraping with keeping a log and write into xml

In [47]:
from IPython.display import HTML, display
def progress(num1, state, d, num2):
    return HTML("""Scraped table number: {n1}
                State: {s}
                District: {dis}
                Tables written in file: {n2}""".format(n1=num1, s=state, dis=d, n2=num2))

In [51]:
import pandas as pd
import xml.etree.ElementTree as ET


# Check log untill which district has been scraped and written in file
last = 0
file = open('log.txt', 'r')
l = []
s = file.readline()
while s:
    l.append(s.strip())
    s = file.readline()
file.close()
if len(l) == 0:
    last = 1
else:
    s = l[-1]
count1 = len(l)
count2 = 0
for i in l:
    if len(i.split(',')[2].strip()) == 3:
        count2 += 1



doc = ET.Element('document')

out = display(progress(count1, "", "", count2), display_id=True)




states = getStates()

for state in states:
    districts = getDistrict(state)
    
    for dis in districts:
        if last == 1:
            try:
                table = getTable(state, dis)
            except:
                b_xml = ET.tostring(doc)
                file = open('weather_tables.xml', 'ab')
                file.write(b_xml)
                file.close()

            count1 += 1
            f = open('log.txt', 'a')
            if(len(table) > 1):
                f.write(state+','+dis+',Yes')
            else:
                f.write(state+','+dis+',No')
            f.write('\n')
            f.close()

            if(len(table) == 1):
                out.update(progress(count1, state, dis, count2))
                continue
            count2 += 1
            
            tb = ET.SubElement(doc, 'table')
            md = ET.SubElement(tb, 'metadata')
            md1 = ET.SubElement(md, 'StateName')
            md1.text = state
            md2 = ET.SubElement(md, 'DistrictName')
            md2.text = dis
            md3 = ET.SubElement(md, 'ForecastRange')
            md3.text = table[1][0] + ' to ' + table[-1][0]

            head = ET.SubElement(tb, 'header')
            text = ''
            for h in table[0]:
                text += h + ' | '
            text = text[:-2]
            head.text = text

            for r in table[1:]:
                row = ET.SubElement(tb, 'row')
                text = ''
                for ele in r:
                    text += ele + ' | '
                text = text[:-2]
                row.text = text
            
            out.update(progress(count1, state, dis, count2))

            
        elif (s.split(',')[0].strip().lower() == state.strip().lower() and s.split(',')[1].strip().lower() == dis.strip().lower()):
            last = 1
            continue

b_xml = ET.tostring(doc)
file = open('weather_tables.xml', 'ab')
file.write(b_xml)
file.close()


Exception ignored in: <function Service.__del__ at 0x00000278E60CC4A0>
Traceback (most recent call last):
  File "c:\Users\tatha\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Users\tatha\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\tatha\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\tatha\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 215, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tatha\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 515, in open
    response = self._open(req, data)
         